# Speculative Decoding

SGLang now provides an EAGLE-based speculative decoding option. The implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

**Note:** Currently, Speculative Decoding in SGLang does not support radix cache.

### Performance Highlights

- Official EAGLE code ([SafeAILab/EAGLE](https://github.com/SafeAILab/EAGLE)): ~200 tokens/s
- Standard SGLang Decoding: ~156 tokens/s
- EAGLE Decoding in SGLang: ~297 tokens/s
- EAGLE Decoding in SGLang (w/ `torch.compile`): ~316 tokens/s

All benchmarks below were run on a single H100.

## EAGLE Decoding

To enable EAGLE-based speculative decoding, specify the draft model (`--speculative-draft`) and the relevant EAGLE parameters:

In [1]:
# EAGLE decoding
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

server_process = execute_shell_command(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algo EAGLE \
    --speculative-draft lmzheng/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.7 --port=30020 --cuda-graph-max-bs 32
"""
)

wait_for_server("http://localhost:30020")

[2025-02-14 02:38:03] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='127.0.0.1', port=30020, mem_fraction_static=0.7, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=True, tp_size=1, stream_interval=1, stream_output=False, random_seed=604677152, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=False, decode_log

[2025-02-14 02:38:33 TP0] Init torch distributed begin.
[2025-02-14 02:38:33 TP0] Load weight begin. avail mem=78.81 GB


[2025-02-14 02:38:35 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.75s/it]


...

 97%|█████████▋| 31/32 [00:24<00:00,  1.89it/s]

100%|██████████| 32/32 [00:24<00:00,  1.29it/s]
[2025-02-14 02:39:15 TP0] Capture cuda graph end. Time elapsed: 24.81 s
[2025-02-14 02:39:15 TP0] max_total_num_tokens=87056, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=4097, context_len=4096


[2025-02-14 02:39:15] INFO:     Started server process [142945]
[2025-02-14 02:39:15] INFO:     Waiting for application startup.
[2025-02-14 02:39:15] INFO:     Application startup complete.
[2025-02-14 02:39:15] INFO:     Uvicorn running on http://127.0.0.1:30020 (Press CTRL+C to quit)
[2025-02-14 02:39:15] INFO:     127.0.0.1:58408 - "GET /v1/models HTTP/1.1" 200 OK


[2025-02-14 02:39:16] INFO:     127.0.0.1:58412 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-14 02:39:16 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-14 02:39:18] INFO:     127.0.0.1:58422 - "POST /generate HTTP/1.1" 200 OK
[2025-02-14 02:39:18] The server is fired up and ready to roll!


In [2]:
import openai

client = openai.Client(base_url="http://127.0.0.1:30020/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-02-14 02:39:23 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-14 02:39:23] INFO:     127.0.0.1:58430 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [3]:
terminate_process(server_process)

### EAGLE Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--cuda-graph-max-bs`:


In [4]:
server_process = execute_shell_command(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algo EAGLE \
    --speculative-draft lmzheng/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.7 \
            --enable-torch-compile --cuda-graph-max-bs 2 --port=30020
"""
)

wait_for_server("http://localhost:30020")

[2025-02-14 02:39:51] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='127.0.0.1', port=30020, mem_fraction_static=0.7, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=True, tp_size=1, stream_interval=1, stream_output=False, random_seed=702775687, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=False, decode_log

[2025-02-14 02:40:21 TP0] Init torch distributed begin.
[2025-02-14 02:40:21 TP0] Load weight begin. avail mem=78.81 GB


[2025-02-14 02:40:23 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.89s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.33s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.42s/it]

[2025-02-14 02:40:26 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.float16, avail mem=66.15 GB
[2025-02-14 02:40:26 TP0] KV Cache is allocated. K size: 21.25 GB, V size: 21.25 GB.
[2025-02-14 02:40:26 TP0] Memory pool end. avail mem=23.46 GB


...

 50%|█████     | 1/2 [00:05<00:05,  5.80s/it]

100%|██████████| 2/2 [00:11<00:00,  5.68s/it]
[2025-02-14 02:42:26 TP0] Capture cuda graph end. Time elapsed: 11.36 s
[2025-02-14 02:42:26 TP0] max_total_num_tokens=87056, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=4097, context_len=4096


[2025-02-14 02:42:26] INFO:     Started server process [144165]
[2025-02-14 02:42:26] INFO:     Waiting for application startup.
[2025-02-14 02:42:26] INFO:     Application startup complete.
[2025-02-14 02:42:26] INFO:     Uvicorn running on http://127.0.0.1:30020 (Press CTRL+C to quit)


[2025-02-14 02:42:27] INFO:     127.0.0.1:46032 - "GET /v1/models HTTP/1.1" 200 OK
[2025-02-14 02:42:27] INFO:     127.0.0.1:46034 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-14 02:42:27 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-14 02:42:27] INFO:     127.0.0.1:46048 - "POST /generate HTTP/1.1" 200 OK
[2025-02-14 02:42:27] The server is fired up and ready to roll!


## Benchmark Script

The following code example shows how to measure the decoding speed when generating tokens:


In [5]:
import time
import requests

tic = time.time()
response = requests.post(
    "http://localhost:30020/generate",
    json={
        "text": "[INST] Give me a simple FastAPI server. Show the python code. [/INST]",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 256,
        },
    },
)
latency = time.time() - tic
ret = response.json()
completion_text = ret["text"]
speed = ret["meta_info"]["completion_tokens"] / latency

print_highlight(completion_text)
print_highlight(f"speed: {speed:.2f} token/s")

[2025-02-14 02:42:32 TP0] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-14 02:42:33 TP0] Decode batch. #running-req: 1, #token: 185, token usage: 0.00, accept len: 4.12, gen throughput (token/s): 26.36, #queue-req: 0


[2025-02-14 02:42:33] INFO:     127.0.0.1:46064 - "POST /generate HTTP/1.1" 200 OK


In [6]:
terminate_process(server_process)